In [ ]:
;pwd

In [ ]:
]activate ..

In [ ]:
using Revise

In [ ]:
using Apophis

In [ ]:
# using TaylorIntegration, LinearAlgebra # Apophis.jl @reexports TaylorIntegration and LinearAlgebra
using Plots, DelimitedFiles, Dates
using Statistics: mean, std
using AstroTime

In [ ]:
using JLD
#recover integration from .jld file
#vars = ["tv_jpl_integ", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
vars = ["tv_jpl_integ"]
filename = string("Apophis_jt.jld")
for i in eachindex(vars)
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])
end

In [ ]:
jpl_radar = readdlm("../Apophis_JPL_data.dat", '\t')

#monostatic mode: check that each receiver and transmitter are the same
@show all( jpl_radar[:,7] .== jpl_radar[:,8] )

#extract transmitter frequencies (Hz)
transmitter_freq = jpl_radar[:,6]*1e6;

#then, get the station codes
station_codes = Int.(jpl_radar[:,7]);

#construct vector of observation times
#####`union` removes repeated elements
#NOTE: resulting vector does not include t0
df_jpl = "y-m-d H:M:S"
tv_jpl = Dates.datetime2julian.(  DateTime.(jpl_radar[:,2], df_jpl))

#check that time vectors from NEODyS and JPL are the same
#tv_neodys_obs == union(t0, tv_jpl[tv_jpl .> t0])
tv_jpl_utc = DateTime.(jpl_radar[:,2], df_jpl)[8:end]
tv_jpl_utc_julian = Dates.datetime2julian.(tv_jpl_utc)
tv_jpl_utc_julian_unrepeated = union(julian(UTCEpoch(TDBEpoch(t0, origin=:julian))).Δt, tv_jpl_utc_julian);

In [ ]:
#get index vector of observation times
#takes into account only times in `tv_jpl_integ` greater than t0
radar_jpl_obs_ind = map(y->findfirst(x->x == y, tv_jpl), tv_jpl_utc_julian_unrepeated[2:end]);

In [ ]:
#check that we got `radar_obs_ind` right
tv_jpl[radar_jpl_obs_ind] == tv_jpl_utc_julian_unrepeated[2:end]

In [ ]:
#get set of indices in jpl_radar[:,5] which are, resp., delay and Doppler observations (>t0)
del_ind = findall(y->y=="us", jpl_radar[:,5])[3:end]
dop_ind = findall(y->y=="Hz", jpl_radar[:,5])[6:end];

Let $\vec R$ denote the geocentric position of the observing station, $\vec \rho$ the topocentric position of the asteroid relative to the same station,  $\vec r_\mathrm{a}$ the geocentric position of the asteroid and $\vec r_\mathrm{E}$ the geocentric position of the asteroid. Then we have

$$
\vec r_\mathrm{a} =\vec r_\mathrm{E} + \vec R + \vec \rho
$$

Thus, the instantaneous topocentric range $\rho = |\vec \rho|$ of the asteroid may be computed as

$$
\rho = |\vec r_\mathrm{a} - \vec r_\mathrm{E} - \vec R| = \sqrt{(x_\mathrm{a} - x_\mathrm{E} - X)^2+(y_\mathrm{a} - y_\mathrm{E} - Y)^2+(z_\mathrm{a} - z_\mathrm{E} - Z)^2}
$$

And the instantaneous range rate $\dot \rho$ may be computed as:

$$
\dot \rho = \frac{1}{\rho}\vec{\rho} \cdot \dot{\vec{\rho}}
$$

#### Questions
- Is TDB the actual independent variable in the integration of the Solar System's equations of motion? ANSWER: YES
- Where can I get the integrated values (evolution?) of the orientation of the Moon, so that I do not have to integrate that from the beginning (1969)?

# Delay/Doppler observations reduction

In [ ]:
niters = 3 # number of iterations in time delay recursive process
rad_ind = radar_jpl_obs_ind[2]

In [ ]:
station_codes[rad_ind]

tdelay, dshift = delay_doppler(
    xv1[3, :],
    station_codes[rad_ind],
    tv_jpl[rad_ind],
    transmitter_freq[rad_ind],
    niters
)

In [ ]:
tdelay_jpl, dshift_jpl = delay_doppler_jpleph(
    station_codes[rad_ind],
    tv_jpl[rad_ind],
    transmitter_freq[rad_ind],
    niters
)

tdelay

In [ ]:
tdelay_jpl

102682986.05 - tdelay() #O-C

In [ ]:
jpl_radar[del_ind[1],3]

In [ ]:
tdelay_jpl

In [ ]:
jpl_radar[del_ind[1],3] - tdelay_jpl #O-C, JPL

dshift

57880.250 - dshift() #O-C

In [ ]:
57880.250 - dshift_jpl #O-C, JPL

#get time-delay, Doppler-shift values from integration

tdelay_v1 = Array{Taylor1{Float64}}(undef, length(tv_jpl_integ)-1)
dshift_v1 = Array{Taylor1{Float64}}(undef, length(tv_jpl_integ)-1)

for i in eachindex(tv_jpl_utc_julian_unrepeated[2:end])
    j = radar_jpl_obs_ind[i]
    tdelay_v1[i], dshift_v1[i] = delay_doppler(
        xv1[i+1, :],
        station_codes[j],
        tv_jpl[j],
        transmitter_freq[j],
        niters
    )
end


In [ ]:
# get time-delay, Doppler-shift values from integration

tdelay_JPL_v1 = Array{Float64}(undef, length(tv_jpl_utc_julian_unrepeated)-1)
dshift_JPL_v1 = Array{Float64}(undef, length(tv_jpl_utc_julian_unrepeated)-1)

for i in eachindex(tv_jpl_utc_julian_unrepeated[2:end])
    j = radar_jpl_obs_ind[i]
    tdelay_JPL_v1[i], dshift_JPL_v1[i] = delay_doppler_jpleph(
        station_codes[j],
        tv_jpl[j],
        transmitter_freq[j],
        niters
    )
end

# Time-delay / Doppler shift plots

In [ ]:
#get corresponding set of indices in tv_jpl_integ
tv_del_ind = findall(x->in(x, tv_jpl[del_ind]), tv_jpl_utc_julian_unrepeated) .- 1
tv_dop_ind = findall(x->in(x, tv_jpl[dop_ind]), tv_jpl_utc_julian_unrepeated) .- 1;

In [ ]:
#check that indices are correct
union(sort(vcat(tv_del_ind, tv_dop_ind))) == collect(1:29)

#TaylorIntegration.jl: Observed minus computed (O-C) residuals

#absolute (TaylorIntegration.jl)
residual_td = Float64.(jpl_radar[del_ind,3]) - tdelay_v1[tv_del_ind] # (usec)
residual_ds = Float64.(jpl_radar[dop_ind,3]) - dshift_v1[tv_dop_ind] # (Hz)

#relative (TaylorIntegration.jl)
rel_res_td = residual_td ./ Float64.(jpl_radar[del_ind,3]) # (usec)
rel_res_ds = residual_ds ./ Float64.(jpl_radar[dop_ind,3]); # (Hz)

In [ ]:
# Observed minus computed (O-C) residuals

# absolute (JPL DE430+s199)
residual_JPL_td = Float64.(jpl_radar[del_ind,3]) - tdelay_JPL_v1[tv_del_ind] # (usec)
residual_JPL_ds = Float64.(jpl_radar[dop_ind,3]) - dshift_JPL_v1[tv_dop_ind] # (Hz)

# relative (JPL DE430+s199)
rel_res_JPL_td = residual_JPL_td ./ Float64.(jpl_radar[del_ind,3]) # (usec)
rel_res_JPL_ds = residual_JPL_ds ./ Float64.(jpl_radar[dop_ind,3]); # (Hz)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    tdelay_JPL_v1[tv_del_ind],
    label="predicted (JPL DE430+s199)",
    marker=:star4
)
#scatter!(
#    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
#    tdelay_v1[tv_del_ind](),
#    label="predicted (TaylorIntegration.jl)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    Float64.(jpl_radar[del_ind,3]),
    label="observed (JPL)",
    legend=:topleft,
    marker=:cross,
    yerror=Float64.(jpl_radar[del_ind,4])
)
title!("Time delay vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay [microseconds]")
#xlims!(1550, 1580)
#ylims!(9e7,1.1e8)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    dshift_JPL_v1[tv_dop_ind],
    label="predicted (JPL DE430+s199)",
    marker=:star4
)
#scatter!(
#    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
#    dshift_v1[tv_dop_ind](),
#    label="predicted (TaylorIntegration.jl)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    Float64.(jpl_radar[dop_ind,3]),
    label="observed (JPL)",
    legend=:topright,
    marker=:cross,
    yerror=Float64.(jpl_radar[dop_ind,4])
)
title!("Doppler shift vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift [Hz]")
#xlims!(1600,1640)

In [ ]:
plot(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    #299792.458*1e-6residual_JPL_td, yerror = 299792.458*1e-6Float64.(jpl_radar[del_ind,4]),
    residual_JPL_td, yerror=Float64.(jpl_radar[del_ind,4]),
    marker=:xcross,
    label = "time delay (O-C) (JPL DE430+s199)",
    legend = :bottomright
)
#scatter!(
#    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
#    residual_td(),
#    yerror=Float64.(jpl_radar[del_ind,4]),
#    marker=:cross,
#    label = "time delay (O-C) (TaylorIntegration.jl)"
#)
title!("Time delay O-C residuals")
xlabel!("t-t0 [Julian days]")
#ylabel!("Total time delay resid. (O-C) [km]")
ylabel!("Total time delay resid. (O-C) [us]")
#ylims!(0,60)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    residual_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4]),
    marker=:xcross,
    label = "Doppler shift (O-C) (JPL DE430+s199)",
    legend=:topright
)
#scatter!(
#    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
#    residual_ds(),
#    yerror=Float64.(jpl_radar[dop_ind,4]),
#    marker=:cross,
#    label = "Doppler shift (O-C) (TaylorIntegration.jl)",
#    legend=:bottomright
#)
title!("Doppler shift residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
#xlims!(1540,1580)

scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    (299792458*1e-9/2)*residual_JPL_td,
    yerror=(299792458*1e-9/2)*Float64.(jpl_radar[del_ind,4]),
    marker=:xcross,
    label = "time delay (O-C) (JPL DE430+s199)"
)
title!("Time delay O-C residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay resid. (O-C) [km]")

scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    residual_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4]),
    marker=:xcross,
    label = "Doppler shift (O-C) (JPL DE430+s199)"
)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    rel_res_td(),
    yerror=Float64.(jpl_radar[del_ind,4])./Float64.(jpl_radar[del_ind,3]),
    label="Relative time delay O-C resid. (TaylorIntegration)",
    marker=:cross,
    legend=:topright
)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    rel_res_JPL_td,
    yerror=Float64.(jpl_radar[del_ind,4])./Float64.(jpl_radar[del_ind,3]),
    label="Relative time delay residual (DE430+s199)",
    marker=:xcross,
    legend=:topright
)
title!("Time delay relative O-C resid.")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    rel_res_ds(),
    yerror=Float64.(jpl_radar[dop_ind,4])./Float64.(jpl_radar[dop_ind,3]),
    label="Relative Doppler shift O-C resid. (TI)",
    legend=:bottomright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    rel_res_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4])./Float64.(jpl_radar[dop_ind,3]),
    label="Relative Doppler shift O-C resid. (JPL)",
    legend=:bottomright,
    marker=:xcross
)
title!("Doppler shift relative residuals")
xlabel!("t-t0 [Julian days]")

plot(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    rel_res_JPL_td,
    yerror=Float64.(jpl_radar[del_ind,4])./Float64.(jpl_radar[del_ind,3]),
    label="Relative time delay residual (DE430+s199)",
    marker=:xcross,
    legend=:topright
)

scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    rel_res_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4])./Float64.(jpl_radar[dop_ind,3]),
    label="Relative Doppler shift O-C resid. (JPL)",
    legend=:bottomright,
    marker=:xcross
)

#check that HORIZONS (s#199) and downloaded SPK file for Apophis are the same
#according to this check, they differ at the end of the evaluated times by ~ 2m in position
tv_spkvhor = range(2454733.5, length=1001, stop=2456367.5)
apophis_spk_arr = Array{Float64}(undef, length(tv_spkvhor), 6)
for i in eachindex(tv_spkvhor)
    apophis_spk_arr[i,:] .= Apophis.apophis_pv(tv_spkvhor[i])
end
apophis_horizons_arr = readdlm("../wld103954.15")
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,1:3])
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,4:6])
;

# Yarkovsky $A_2$ coefficient estimation

In [ ]:
tdelay_v1[1] # τ(A2) = p(A2)

In [ ]:
tdelay_v1[1]-constant_term(tdelay_v1[1]) # δτ(A2) = τ(A2) - τ(A2=0) = p(A2) - τ(A2=0) = p[1]*A2+p[2]*A2^2+... ( p[0]=τ(A2=0) )

In [ ]:
inverse( tdelay_v1[1]-constant_term(tdelay_v1[1]) ) # A2(δτ) = q(δτ) = q[1]*δτ+q[2]*δτ^2+... (q[0]=0)

In [ ]:
td_us = tdelay_v1[tv_del_ind]
ds_Hz = dshift_v1[tv_dop_ind];

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δr) polynomials at each delay observation;
A2_δf_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δvr) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δf_v, residual_ds()); # A2(δf_Doppler) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
scatter(tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0, A2_del_v, leg=false, marker=:cross)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(dt) x 10^14 [au/d^2]")
ylims!(1.1minimum(A2_del_v),0)

In [ ]:
scatter(tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0, A2_dop_v, leg=false, marker=:cross)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(df) x 10^14 [au/d^2]")
ylims!(1.1minimum(A2_dop_v),0)

# Intervals

In [ ]:
using IntervalArithmetic

In [ ]:
residual_td_interval = interval.(
    (jpl_radar[del_ind,3]-jpl_radar[del_ind,4])-tdelay_v1[tv_del_ind](), 
    (jpl_radar[del_ind,3]+jpl_radar[del_ind,4])-tdelay_v1[tv_del_ind]()
)

In [ ]:
residual_ds_interval = interval.(
    (jpl_radar[dop_ind,3]-jpl_radar[dop_ind,4])-dshift_v1[tv_dop_ind](),
    (jpl_radar[dop_ind,3]+jpl_radar[dop_ind,4])-dshift_v1[tv_dop_ind]()
)

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δτ_v, residual_td_interval)

In [ ]:
A2_dop_v_interval = map((x,y)->x(y), A2_δf_v, residual_ds_interval)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    A2_del_v,
    yerror=radius.(A2_del_v_interval),
    leg=false,
    marker=:cross
)
ylims!(minimum(A2_del_v)-1,0)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    A2_dop_v,
    yerror=radius.(A2_dop_v_interval),
    leg=false,
    marker=:cross
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    A2_del_v,
    yerror=radius.(A2_del_v_interval),
    leg=false,
    marker=:cross
)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    A2_dop_v,
    yerror=radius.(A2_dop_v_interval),
    leg=false,
    marker=:cross
)